In [ ]:
from core.helpers.session_helper import SessionHelper
session = SessionHelper().session

In [ ]:
"""
************ CONFIGURATION - PLEASE TOUCH **************
Pipeline Builder configuration: creates configurations from variables specified here!!
This cell will be off in production as configurations will come from the configuration postgres DB.
"""
# config vars: this dataset
config_pharma = "sun" # the pharmaceutical company which owns {brand}
config_brand = "ilumya" # the brand this pipeline operates on
config_state = "enrich" # the state this transform runs in
config_name = "pending_new_enrich" # the name of this transform, which is the name of this notebook without .ipynb

# input vars: dataset to fetch. Recall that a contract published to S3 has a key format branch/pharma/brand/state/name
input_pharma= "sun"
input_brand = "ilumya"
input_state = "enrich"
input_name = "upstream"
input_branch = None # if None, input_branch is automagically set to your working branch

In [ ]:
"""
************ SETUP - DON'T TOUCH **************
Populating config mocker based on config parameters...
"""
import core.helpers.pipeline_builder as builder

ids = builder.build(config_pharma, config_brand, config_state, config_name, session)
transform_id = ids[0]
run_id = ids[1]

In [ ]:
"""
************ SETUP - DON'T TOUCH **************
This section imports data from the configuration database
and should not need to be altered or otherwise messed with. 
~~These are not the droids you are looking for~~
"""
from core.constants import BRANCH_NAME, ENV_BUCKET
from core.helpers.session_helper import SessionHelper
from core.models.configuration import Transformation
from dataclasses import dataclass
from core.dataset_contract import DatasetContract

db_transform = session.query(Transformation).filter(Transformation.id == transform_id).one()

@dataclass
class DbTransform:
    id: int = db_transform.id ## the instance id of the transform in the config app
    name: str = db_transform.transformation_template.name ## the transform name in the config app
    state: str = db_transform.pipeline_state.pipeline_state_type.name ## the pipeline state, one of raw, ingest, master, enhance, enrich, metrics, dimensional
    branch:str = BRANCH_NAME ## the git branch for this execution 
    brand: str = db_transform.pipeline_state.pipeline.brand.name ## the pharma brand name
    pharmaceutical_company: str = db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name # the pharma company name
    publish_contract: DatasetContract = DatasetContract(branch=BRANCH_NAME,
                            state=db_transform.pipeline_state.pipeline_state_type.name,
                            parent=db_transform.pipeline_state.pipeline.brand.pharmaceutical_company.name,
                            child=db_transform.pipeline_state.pipeline.brand.name,
                            dataset=db_transform.transformation_template.name)


# CORE Cartridge Notebook::[Pending New Enrichment]
![CORE Logo](assets/coreLogo.png) 

---
## Keep in Mind
Good Transforms Are...
- **singular in purpose:** good transforms do one and only one thing, and handle all known cases for that thing. 
- **repeatable:** transforms should be written in a way that they can be run against the same dataset an infinate number of times and get the same result every time. 
- **easy to read:** 99 times out of 100, readable, clear code that runs a little slower is more valuable than a mess that runs quickly. 
- **No 'magic numbers':** if a variable or function is not instantly obvious as to what it is or does, without context, maybe consider renaming it.

## Workflow - how to use this notebook to make science
#### Data Science
1. **Document your transform.** Fill out the _description_ cell below describing what it is this transform does; this will appear in the configuration application where Ops will create, configure and update pipelines. 
1. **Define your config object.** Fill out the _configuration_ cell below the commented-out guide to define the variables you want ops to set in the configuration application (these will populate here for every pipeline). 
2. **Build your transformation logic.** Use the transformation cell to do that magic that you do. 
![caution](assets/cautionTape.png)

### Configuration

In [ ]:
""" 
********* VARIABLES - PLEASE TOUCH ********* 
This section defines what you expect to get from the configuration application 
in a single "transform" object. Define the vars you need here, and comment inline to the right of them 
for all-in-one documentation. 
Engineering will build a production "transform" object for every pipeline that matches what you define here.

@@@ FORMAT OF THE DATA CLASS IS: @@@ 

<variable_name>: <data_type> #<comment explaining what the value is to future us>

e.g.

class Transform(DbTransform):
    some_ratio: float
    site_name: str

~~These ARE the droids you are looking for~~
"""

class Transform(DbTransform):
    '''
    YOUR properties go here!!
    Variable properties should be assigned to the exact name of
    the transformation as it appears in the Jupyter notebook filename.
    '''
    trans_id: str
    product: str
    patient: str
    pharm: str
    status_date: str
    ref_date: str
    status: str
    substatus: str
    ic_status: str
    ic_substatus: str
    pending_status: str
    active_status: str
    cancelled_status: str
    pending_new_substatus: str
        
    def load_clean_file(self, df):
        # Convert dates to datetime format
        datetime = '%Y%m%d'
        df[self.status_date] = pd.to_datetime(df[self.status_date], format=datetime, errors='coerce')
        df[self.ref_date] = pd.to_datetime(df[self.ref_date], format=datetime, errors='coerce')

        ## Extract brand from medication
        df[self.product] = df['medication'].apply(lambda x: x.split()[0].strip() if not pd.isnull(x) else x)

        ## Convert status codes to uppercase
        df[self.status] = df[self.status].str.upper()
        df[self.substatus] = df[self.substatus].str.upper()
        
        return df
    
    def pending_new_enrichment(self, df, table_columns):

        (
            df
            .sort_values(
                by=[self.patient, self.pharm, self.product, self.status_date], 
                ascending=[True, True, True, True], inplace=True)
        )

        min_date_df = (
            df
            .groupby([self.patient, self.pharm, self.product])[self.status_date].min()
            .reset_index()
            .rename(columns={self.status_date: 'First_Status_Date'})
        )
        
        df = pd.merge(df, min_date_df, how='left', 
                      on=[self.patient, self.pharm, self.product])
        
        min_ref_date_df = (
            df
            .groupby([self.patient, self.pharm, self.product])[self.ref_date].min()
            .reset_index()
            .rename(columns={self.ref_date: 'First_Ref_Date'})
        )
        df = pd.merge(df, min_ref_date_df, how='left', 
                      on=[self.patient, self.pharm, self.product])
        
        (
            df
            .sort_values(
                by=[self.patient, self.pharm, self.product, self.status_date], 
                ascending=[True, True, True, True], inplace=True)
        )
        
        first_status = (
            df
            .groupby([self.pharm, self.product, self.patient])[self.status].first()
            .reset_index()
            .rename(columns={self.status: 'First_IC_Status'})
        )
        
        first_status['First_Status_Active_Cancelled'] = np.where(
            first_status['First_IC_Status'].isin([self.active_status, self.cancelled_status]), 1, 0)
        
        df = pd.merge(df, first_status, how='left', 
                      on=[self.pharm, self.product, self.patient])
        
        pat_df = df.drop_duplicates(subset=[self.patient, self.pharm, self.product], keep='first')
        
        pat_df['Day_Diff'] = (pat_df['First_Status_Date'] - pat_df['First_Ref_Date']) / np.timedelta64(1, 'D')
        pat_df = pat_df[[self.patient, self.pharm, self.product, 'Day_Diff']]
        
        df = pd.merge(df, pat_df, how='left', 
                      on=[self.pharm, self.product, self.patient])
        to_enrich_df = df[((df['First_Status_Active_Cancelled'] == 1)
                          & (df['Day_Diff'] > 1))]
        
        to_enrich_df = to_enrich_df.drop_duplicates(subset=[self.patient, self.pharm, self.product])

        to_enrich_df.loc[:, self.substatus] = self.pending_new_substatus
        to_enrich_df.loc[:, self.status] = self.pending_status
        #to_enrich_df.loc[:, self.ic_substatus] = self.pending_new_substatus
        #to_enrich_df.loc[:, self.ic_status] = self.pending_status
        to_enrich_df.loc[:, 'heirarchy'] = 'No Status Clarity'
        to_enrich_df.loc[:, self.status_date] = to_enrich_df['First_Ref_Date']
        to_enrich_df.loc[:, self.ref_date] = to_enrich_df['First_Ref_Date']
        to_enrich_df.loc[:, self.trans_id] = 'ic_' + to_enrich_df[self.trans_id].astype(str) + '_pn'
        to_enrich_df.loc[:, 'enrichment'] = 'pending new'
        #to_enrich_df = to_enrich_df[table_columns]
        df = df.append(to_enrich_df)
        #df = df[table_columns]
        return (df)

transform = Transform()

In [ ]:
## Please place your value assignments for development here!!
## This cell will be turned off in production and Engineering will set to pull from the configuration application instead
## For the last example, this could look like...
## transform.some_ratio = 0.6
## transform.site_name = "WALGREENS"
if input_pharma == "sun":
    transform.trans_id = 'pharm_transaction_id'
    transform.patient = 'msa_patient_id'
    transform.pharm = 'pharm_code'
    transform.status_date = 'status_date'
    transform.ref_date = 'ref_date'
    transform.status =  'status_code'
    transform.substatus =  'sub_status'
else:
    transform.trans_id = 'pmcTransactionId'
    transform.patient = 'pmcPatientId'
    transform.pharm = 'pharmName'
    transform.status_date = 'statusDate'
    transform.ref_date = 'refDate'
    transform.status =  'statusCode'
    transform.substatus =  'subStatus'
transform.product = 'product'
transform.ic_status = 'integrichain_status'
transform.ic_substatus = 'integrichain_sub_status'
transform.pending_status = 'PENDING'
transform.active_status = 'ACTIVE'
transform.cancelled_status = 'CANCELLED'
transform.pending_new_substatus = 'NEW'

### Description
What does this transformation do? be specific.

![what does your transform do](assets/what.gif)

This transform handles the situations where the first status is active or cancelled and the first status date received exceeds the one day grace period. These instances are handled by making the first status date also the first referral date and making this new row a pending new status. Will finish with more rows than started with.

### Transformation

In [ ]:
"""
************ FETCH DATA - TOUCH, BUT CAREFULLY **************
This cell will be turned off in production, as the input_contract will be handled by the pipeline.
"""

if not input_branch:
    input_branch = BRANCH_NAME
input_contract = DatasetContract(branch=input_branch, state=input_state, parent=input_pharma, child=input_brand, dataset=input_name)
run_filter = []
# run_filter.append(dict(partition="run_id", comparison="==", values=[1]))
# IF YOU HAVE PUBLISHED DATA MULTIPLE TIMES, uncomment the above line and change the int to the run_id to fetch.
# Otherwise, you will have duplicate values in your fetched dataset!
final_dataframe = input_contract.fetch(filters=run_filter)

In [ ]:
import numpy as np
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
from s3fs import S3FileSystem
from core.secret import Secret
from s3parq import fetch
from random import getrandbits

run_id = 4

ingest_contract = DatasetContract(
    branch='sun-extract-validation',
    parent='sun',
    child='ilumya',
    state='ingest',
    dataset='symphony_health_association_ingest_column_mapping'
)

run_filter = [{'partition':'__metadata_run_id', 'comparison':'==', 'values':[run_id]}]

df = fetch(bucket=ingest_contract.bucket, key=ingest_contract.key, filters=run_filter)
df.status_date = df.status_date.str[:8].astype(str)
df.ref_date = df.ref_date.str[:8].astype(str)

In [ ]:
list(df.columns)

In [ ]:
def pandas_from_parquet_s3(file_path):
    
    s3 = S3FileSystem()
    df = (
        pq
        .ParquetDataset(file_path, filesystem=s3)
        .read_pandas()
        .to_pandas()
    )
    
    return df


def pandas_from_database(query, *args):
    
    connection = mysql.connect(
        host=secret.host, 
        user=secret.user, 
        passwd=secret.password, 
        port=secret.port, 
        charset='utf8'
    )
    
    query = query.format(*args)
    df = pd.read_sql(sql=query, con=connection)
    connection.close()
    
    return df


def pandas_to_parquet_s3(df, partition, file_path):
    
    s3 = S3FileSystem()
    table = pa.Table.from_pandas(df)
    
    pq.write_to_dataset(table, file_path, partition_cols=partition, filesystem=s3)

In [ ]:
### Use the variables above to execute your transformation. the final output needs to be a variable named final_dataframe
columns = list(df.columns).append('heirarchy')
df = transform.load_clean_file(df)
final_dataframe= transform.pending_new_enrichment(df, columns)
final_dataframe.head()

In [ ]:
len(df)

In [ ]:
len(final_dataframe)

## Unit Testing

In [ ]:
import unittest

def length_status(final_dataframe,df):
    """
    Make sure all rows from original df are maintained in final,
    This is a test:
    >>> length_status(final_dataframe,df)
    True
    """
    return len(final_dataframe[final_dataframe.pharm_transaction_id.isin(df.pharm_transaction_id)]) == len(df)
    
def check_new_rows(final_dataframe, df):
    """
    Make sure the new rows in final all have a pending status and equal ref and status dates
    This is a test:
    >>> check_new_rows(final_dataframe, df)
    True
    """
    worked =final_dataframe[~(final_dataframe['pharm_transaction_id'].isin(df['pharm_transaction_id']))]
    return ((all(worked['status_code']=='PENDING')) & (worked['ref_date'].isin(worked['status_date']))) 

class TestNotebook(unittest.TestCase):

    def test_length_status(self):
        self.assertEqual(length_status(final_dataframe,df),True)
    
    def test_check_new_rows(self):
        self.assertEqual(check_new_rows(final_dataframe, df),True)
    
unittest.main(argv=[''], verbosity=2, exit=False)

### Publish

In [ ]:
## that's it - just provide the final dataframe to the var final_dataframe and we take it from there
transform.publish_contract.publish(final_dataframe, run_id, session)
session.close()